Needed imports and utility

In [ ]:
import os
import ansys.speos.script as script
import ansys.speos.core as core

tests_data_path = os.path.join(os.path.join(os.path.abspath(''), os.path.pardir), "assets")

def clean_all_dbs(speos_client: core.SpeosClient):
    for item in (
        speos_client.jobs().list()
        + speos_client.scenes().list()
        + speos_client.simulation_templates().list()
        + speos_client.sensor_templates().list()
        + speos_client.source_templates().list()
        + speos_client.intensity_templates().list()
        + speos_client.spectrums().list()
        + speos_client.vop_templates().list()
        + speos_client.sop_templates().list()
        + speos_client.parts().list()
        + speos_client.bodies().list()
        + speos_client.faces().list()
    ):
        item.delete()

tests_data_path = os.path.join(os.path.join(os.path.abspath(''), os.path.pardir), "assets")

Create connection with speos rpc server

In [ ]:
speos = core.Speos(host="localhost", port=50051)
clean_all_dbs(speos.client)

New project

In [ ]:
p = script.Project(speos=speos)
print(p)

# PreRequisites

Sensor, source, root part are prerequisites.

In [ ]:
root_part = p.create_root_part()
root_part.create_body(name="Body.1").create_face(name="Face.1").set_vertices([0, 1, 2, 0, 2, 2, 1, 2, 2])\
                                                               .set_facets([0, 1, 2])\
                                                               .set_normals([0, 0, 1, 0, 0, 1, 0, 0, 1])
root_part.commit()

sensor1 = p.create_sensor(name="Irradiance.1")
sensor1.set_irradiance().set_type_colorimetric() # colorimetric or spectral so that the sensor can be used both in direct and inverse simulation
sensor1.commit()

source1 = p.create_source(name="Surface.1")
source1.set_surface().set_exitance_constant(geometries=[(script.GeoRef.from_native_link(geopath="Body.1/Face.1"), True)])
source1.set_surface().set_spectrum().set_blackbody() # blackbody so that the source can be used both in direct and inverse simulation
source1.commit()



# Create
Create locally  
The mention "local: " is added when printing the simulation

In [ ]:
simulation1 = p.create_simulation(name="Simulation.1")
simulation1.set_direct()
simulation1.set_sensor_paths(["Irradiance.1"]).set_source_paths(["Surface.1"])
print(simulation1)

Push it to the server.  
Now that it is committed to the server, the mention "local: " is no more present when printing the simulation1.

In [ ]:
simulation1.commit()
print(simulation1)

Another example by setting several characteristics

In [ ]:
simulation2 = p.create_simulation(name="Simulation.2")

simulation2_direct = simulation2.set_direct()
simulation2_direct  .set_ambient_material_file_uri(uri=os.path.join(tests_data_path, "AIR.material"))\
                    .set_colorimetric_standard_CIE_1964()\
                    .set_weight_none()\
                    .set_geom_distance_tolerance(0.01)\
                    .set_max_impact(200)\
                    .set_fast_transmission_gathering()\
                    .set_dispersion(False)
simulation2.set_sensor_paths(["Irradiance.1"]).set_source_paths(["Surface.1"]).commit()
print(simulation2)

## Default values
Some default values are available when applicable in every methods and class.

# Read
## Simulation Instance
A mention "local: " is added if it is not yet committed to the server

In [ ]:
print(simulation1)

# Project

In [ ]:
print(p)

# Update
Tipp: if you are manipulating a simulation already committed, don't forget to commit your changes.  
If you don't, you will still only watch what is committed on the server.

In [ ]:
simulation1.set_direct().set_ambient_material_file_uri(uri=os.path.join(tests_data_path, "AIR.material"))
simulation1.commit()
print(simulation1)

## Reset
Possibility to reset local values from the one available in the server.

In [ ]:
simulation1.set_inverse() # set inverse but no commit
simulation1.reset() # reset -> this will apply the server value to the local value
simulation1.delete() # delete (to display the local value with the below print)
print(simulation1)

# Delete
Once the data is deleted from the server, you can still work with local data and maybe commit later.

In [ ]:
simulation2.delete()
print(simulation2)

In [ ]:
print(p)

In [ ]:
simulation1.delete()

# More content
## Inverse Simulation 

In [ ]:
simulation3 = p.create_simulation(name="Simulation.3")
simulation3.set_inverse()
simulation3.set_sensor_paths(sensor_paths=["Irradiance.1"]).set_source_paths(source_paths=["Surface.1"]).commit()
print(simulation3)

## Interactive

In [ ]:
simulation4 = p.create_simulation(name="Simulation.4")
simulation4.set_interactive()
simulation4.set_source_paths(source_paths=["Surface.1"]).commit()
print(simulation4)

# Cleanup

In [ ]:
clean_all_dbs(speos.client)